# Universidad Distrital Francisco José de Caldas
## Class: Advanced Image Processing - Remote Sensing with Python (Basic Activities)
### Por: David Alonso Rueda Rodríguez

### 2. Crear Imágenes Stack

#### Objetivo de la actividad:
- Comparar el rendimiento de 3 rutinas para concatenar las bandas de imágenes satelitales.

#### Librerías requeridas:
- **rasterio** (librería para comparación)
- **gdal** (librería para comparación)
- **earthpy** (librería para comparación)
- datetime

#### Fuente de datos:

https://gis.stackexchange.com/questions/223910/using-rasterio-or-gdal-to-stack-multiple-bands-without-using-subprocess-commands

In [4]:
import rasterio as ras
import glob
import json
import os
import re
import earthpy as et
import earthpy.spatial as es
from osgeo import gdal
from datetime import datetime

In [5]:
def time():
    from datetime import datetime as dt
    now = dt.now()
    timestamp = dt.timestamp(now)
    return timestamp

## Preparación de datos de prueba

### Determinación de Resolución espacial para bandas

Es necesario hacer una identificación de la resolución espacial y con este dato hacer la concatenacióno de las imágenes (layer stack), en caso de realizar la concatenación con todas las bandas estás serán remuestreadas a la menor resolución espacial disponible.

#### Obtener datos desde directorio de trabajo

In [15]:
# Obtener archivos jp2 en el directorio de interés
url_base = 'src/Sentinel/'
images = glob.glob(url_base + '*.jp2')

#### Identificación de resolución espacial para las bandas de la imagen satelital

In [7]:
# Resolución espacial de las bandas en la imagen satelital
bandas = []
res = []
print('Nombre Arhivo'+ '\t'*3 + 'Res. Espacial')
for image in images:
    with ras.open(image) as src:
        #a1 = os.path.splitext(os.path.basename(image))[0]
        a1 = os.path.basename(image)
        print('{}\t\t{:.0f}' .format(a1, src.profile['transform'][0], (-1)*src.profile['transform'][4]))
        bandas.append(a1)
        res.append(str(src.profile['transform'][0]) + ', ' + str(src.profile['transform'][2]))

Nombre Arhivo			Res. Espacial
T18NVL_20200104T152639_B01.jp2		60
T18NVL_20200104T152639_B02.jp2		10
T18NVL_20200104T152639_B03.jp2		10
T18NVL_20200104T152639_B04.jp2		10
T18NVL_20200104T152639_B05.jp2		20
T18NVL_20200104T152639_B06.jp2		20
T18NVL_20200104T152639_B07.jp2		20
T18NVL_20200104T152639_B08.jp2		10
T18NVL_20200104T152639_B08A.jp2		20
T18NVL_20200104T152639_B09.jp2		60
T18NVL_20200104T152639_B10.jp2		60
T18NVL_20200104T152639_B11.jp2		20
T18NVL_20200104T152639_B12.jp2		20


### Resumen de datos para trabajarse

#### - 10 metros resolución espacial

- T18NVL_20200104T152639_B02: 10
- T18NVL_20200104T152639_B03: 10
- T18NVL_20200104T152639_B04: 10
- T18NVL_20200104T152639_B08: 10

#### - 20 metros resolución espacial

- T18NVL_20200104T152639_B05: 20
- T18NVL_20200104T152639_B06: 20
- T18NVL_20200104T152639_B07: 20
- T18NVL_20200104T152639_B08A: 20
- T18NVL_20200104T152639_B11: 20
- T18NVL_20200104T152639_B12: 20

#### - 60 metros resolución espacial

- T18NVL_20200104T152639_B01: 60
- T18NVL_20200104T152639_B09: 60
- T18NVL_20200104T152639_B10: 60

In [16]:
# Discriminación de datos por resolución espacial
sentinel_10 = [
    'T18NVL_20200104T152639_B02.jp2', 'T18NVL_20200104T152639_B03.jp2',
    'T18NVL_20200104T152639_B04.jp2', 'T18NVL_20200104T152639_B08.jp2'
]
sentinel_20 = [
    'T18NVL_20200104T152639_B05.jp2', 'T18NVL_20200104T152639_B06.jp2', 
    'T18NVL_20200104T152639_B07.jp2', 'T18NVL_20200104T152639_B08A.jp2', 
    'T18NVL_20200104T152639_B11.jp2', 'T18NVL_20200104T152639_B12.jp2'
]
sentinel_60 = [
    'T18NVL_20200104T152639_B01.jp2', 'T18NVL_20200104T152639_B09.jp2',
    'T18NVL_20200104T152639_B10.jp2'
]

In [17]:
# Agregación de url_base con listado de archivos
sentinel_10m = []
sentinel_20m = []
for i in sentinel_10:
    sentinel_10m.append(url_base + i)
for i in sentinel_20:
    sentinel_20m.append(url_base + i)  

In [18]:
# Listado de archivos para prueba
sentinel_10m

['src/Sentinel/T18NVL_20200104T152639_B02.jp2',
 'src/Sentinel/T18NVL_20200104T152639_B03.jp2',
 'src/Sentinel/T18NVL_20200104T152639_B04.jp2',
 'src/Sentinel/T18NVL_20200104T152639_B08.jp2']

## Datos de prueba

### Layer Stack con librería GDAL
Esta prueba permite identificar el tiempo empleado en la creación del layer stack con la librería **GDAL**

In [19]:
#
# Medición de tiempo inicial.
timestamp = time()
#
# Inicializar rutas relativas para almacenamiento de archivos
outvrt = os.path.join(url_base, 'gdal/Sentinel_10m.vrt')
outtif = os.path.join(url_base, 'gdal/Sentinel_10m.tif')
#
# Crear archivo
outds = gdal.BuildVRT(outvrt, sentinel_10m, separate=True)
outds = gdal.Translate(outtif, outds)
#
# Medición de tiempo final
timestamp2 = time()
#
# Tiempo empleado en la actividad
print("Layer Stack Finalizado.\nDuración de la actividad: {:.2f} segundos" .format(timestamp2-timestamp))

Layer Stack Finalizado.
Duración de la actividad: 131.01 segundos


### Layer Stack con Earthpy

Esta prueba permite identificar el tiempo empleado en la creación del layer stack con la librería **Earthpy**

In [20]:
#
# Rutina para medicación de tiempo empleado.
now = datetime.now()
timestamp = datetime.timestamp(now)

#
# Inicialización de rutas de trabajo
et.io.HOME = '/home/jovyan/work/Ciencia/src/'
raster_out_path = os.path.join(et.io.HOME, "Sentinel/earthpy/")

array, raster_prof = es.stack(sentinel_10m, out_path=raster_out_path + 'Sentinel_Out.jp2')

#
# Rutina para medicación de tiempo empleado.
now = datetime.now()
timestamp2 = datetime.timestamp(now)
print("Layer Stack Finalizado.\nDuración de la actividad: {:.2f} segundos" .format(timestamp2-timestamp))

Layer Stack Finalizado.
Duración de la actividad: 380.59 segundos


### Layer Stack con Librería Rasterio
Esta prueba permite identificar el tiempo empleado en la creación del layer stack con la librería **Rasterio**

In [65]:
#
# Rutina para medicación de tiempo empleado.
now = datetime.now()
timestamp = datetime.timestamp(now)

# Crear Layer stack!
file_list = glob.glob('src/Sentinel/*.jp2')
# Read metadata of first file
with ras.open(file_list[0]) as src0:
    meta = src0.meta

# Update meta to reflect the number of layers
meta.update(count = len(file_list))

# Read each layer and write it to stack
with ras.open('src/Sentinel/rasterio/Sentinel.tif', 'w', **meta) as dst:
    for id, layer in enumerate(file_list, start=1):
        with ras.open(layer) as src1:
            dst.write_band(id, src1.read(1))

now = datetime.now()
timestamp2 = datetime.timestamp(now)
print("Layer Stack Finalizado.\nDuración de la actividad: {:.2f} segundos" .format(timestamp2-timestamp))